# Treatment of Covid-19 with Remdesivir - Systematic Review and Meta-Analysis

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
from urllib.request import Request, urlopen
from urllib.error import HTTPError

## Search pubmed for appropriate studies
1. Click on this [link](https://pubmed.ncbi.nlm.nih.gov/)
2. Type "(((((((covid[Title/Abstract]) OR (corona[Title/Abstract])) OR (sars-cov-2[Title/Abstract])) OR (sars cov 2[Title/Abstract])) AND (remdesivir[Title])) NOT (Meta Analysis[Title])) NOT (Meta-Analysis[Title])) NOT (Review[Title])" into search box
3. On the left hand side check "Free Full Text"
4. Click on save and select all results in selection and format Pubmed
5. Click on create file and save it under the name search_results.txt

## Automated Filtering
### Filter for Randomized Control Trials
We only want to include randomized control trials in our meta-analysis. We are using a machine learning tool called robotsearch to filter out only the studies from the pubmed search which used a ranodmized control trial design.
1. Move the file search_results.txt into the robotsearch directory
2. Open Anaconda
3. `cd` your way into the robotsearch directory within the overall covid_systematic_review directory you downloaded from github
4. If the environment is not activated type `conda activate covid_review`.
5. Run `python setup.py install`.
6. Run `robotsearch search_results.txt -p` Don't forget the -p-flag, or the results will contain too many false positive you will have to manually check later.

The results are saved in the file search_reults_robotviewer_RCTs.txt. Let's look at the result

In [2]:
from robotsearch.parsers import ris
file_input = "robotsearch/search_results.txt"
file_result = "robotsearch/search_results_robotreviewer_RCTs.txt"
with open(file_input, 'r', encoding="utf8") as f:
    inp = ris.load(f)
with open(file_result, 'r', encoding="utf8") as f:
    result = ris.load(f)
print("The inital search result has {} articles".format(len(inp)))
print("{} articles were classified as rcts".format(len(result)))

other non numbered
other non numbered
The inital search result has 671 articles
25 articles were classified as rcts


In [3]:
#extract PMC - they are found in the 'PMC' key.
pmcs = []
for i in range(len(result)):
    if "PMC" in result[i].keys():
        id_raw = result[i]['PMC'][0]
        pmc = id_raw.strip()
        pmcs.append(pmc)

print("{} studies have a pmc-identifier and can be accessed through pubmed open-access".format(len(pmcs)))

25 studies have a pmc-identifier and can be accessed through pubmed open-access


### Filter for Outcome
Let's reduce the number even further by checking if the article contains necessary information. Here we only want articles that use time to clinical improvement. The article has to include the meassure hazard-ratio or rate-ratio!

In [62]:
site= "https://www.ncbi.nlm.nih.gov/pmc/articles/{}/"
hdr = {'User-Agent': 'Mozilla/5.0'}
regex_mortality = r"method.+(?:(?:mortality|death)|(?:time to(?:clinical improvement|recovery)))" # the outcome meassure must come after the word method, so it is likey mentioned in the methods or results section
regex_hr = r"method.+(?:hazard|rate)[\s-]ratio"
pmcs_outcome = []
for i in range(len(pmcs)):
    print(i)
    url = site.format(pmcs[i])
    req = Request(url,headers=hdr)
    try:
        page = urlopen(req)
        soup = BeautifulSoup(page)
        if re.search(regex_mortality, soup.prettify(), re.DOTALL|re.IGNORECASE) and re.search(regex_hr, soup.prettify(), re.DOTALL|re.IGNORECASE):
            print(True)
            pmcs_outcome.append(pmcs[i])
    except HTTPError:
        print("httperror")

0
True
1
True
2
True
3
True
4
True
5
True
6
True
7
8
9
10
True
11
12
True
13
14
15
16
True
17
True
18
True
19
20
21
True
22
23
24
True


In [63]:
print("{} articles contain mortality/death or time to clinical improvement or recovery and propper outcome meassure".format(len(pmcs_outcome)))
with open("papers_html.txt", "w") as f:
    for pmc in pmcs_outcome:
        f.write("%s\n" % "https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(pmc))

14 articles contain mortality/death or time to clinical improvement or recovery and propper outcome meassure


In [12]:
screening_results = pd.DataFrame(columns = ['pmc', 'non_retracted', 'randomized_controlled', 'adults', 'infected', 'remdesivir_only', 'propper_outcome'])
screening_results['pmc'] = np.array(pmcs_outcome)
screening_results.to_csv("screening_results.csv", index = False, sep = ";")
for i in range(len(pmcs_outcome)):
    print("https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(pmcs_outcome[i]))

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7262788/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7745180/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8757570/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7377062/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8863204/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8490137/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8279143/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8523116/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7538864/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7190303/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9017992/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7442954/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8439621/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9278225/


## Manual Screening of Articles
Now that we filtered out the promising studies, we must manually check for their eligibility. Even though we automated the classification as randomized controll trial, we must check for errors. We follow the following protocol. Open each of the htmls above by clicking on the link or open the file papers_html.txt. Apply the protocol below to each study individually. Open the file screening_results.csv in a csv-editor of your choice to record your results. Note that the separator is semicolon. Type 1 into the cell if criterion is met and 0 if it is not met. Stop checking the other criteria once you coded one criterion as 0 and continue to the next. Note that initially when this protocol was created, studies had to include a placebo-group. This turned out to too strict a criterion, because only few studies fulfilled all criteria. Therefore we decided to include open-label studies.
### Not retracted
If the article has been retracted, it shows in a red box on top of the webpage. It's impossible to miss.
### Randomized Controlled Trial
Randomized Clinical Trials are those in which patients are assigned randomly to a group receiving treatment with Remdesivir and a placebo-group. Check the abstract and methods secion to check if the papers are randomized control trials. Don't include studies, in which both experimental and control group involves Remdesivir but different dossage. Keywords indicating that a study is not an RCT are: "retrospective" study or analysis or observational cohort study. Note that randomizatin between remdesivir and control must take place. There are studies, in which participants are randomized between two different dosages of remdesivir.
### Majority Aduls. 
Check if proportion of age above 18 is >95%
### Participants infected 
Participants must be infected with Covid-19. No studies analysing Remdesivir as a prevention treatment should be included.
### Remdesivir only 
The only difference between treatment- and control-group should be whether patients received remdesivir or not. Control groups can be placebo or standard-care. If multiple treatments are tested, the study must include a comparison of remdesivir with a placebo or standard care group. If both groups receive remdesivir, but in different dosages you should exclude the study. If there are multiple treatments tested in the same study, there must be a propper comparison of remdesivir and a control-group.
### Propper outcome
To assess the effectiveness of treatment with Remdesivir compared to the placebo or standard-care, there must be a propper effect-meassure reported. 
Check if at least one of the outcomes "time to recovery" or "time to clinical improvement" or equivalent meassures has one of these meassures associated with it: odds-ratio, hazard-ratio, rate-ratio. Keep in mind that what the authors declare as primary outcome might not be the primary outcome for this study. You might even have to look into the appendices to retrieve it.
### Propper outcome
To assess the effectiveness of treatment with Remdesivir compared to the placebo or standard-care, there must be a propper effect-meassure reported. Check if there is a ratio (hazard-, rate-, or odds-ratio) comparing mortality in the remdisivir group to mortality in the control group and/or a ratio comparing time-to recovery or clinical improvement.

## Extract eligable studies
We are going to use the studies that fulfill the above criteria. Execute the cell below to get the studies to be included in the final analysis, after you have comleted and saved the screening of articles.

In [66]:
# Please make sure to save your screening_results.csv properly. In Germany the default seperator is semicolon.
df_screening = pd.read_csv("screening_results.csv",sep=";")
df_screening = df_screening.fillna(0)
df_screening["eligable"] = (df_screening.iloc[:,1:].sum(axis = 1)/6).astype(int).astype(bool)
eligable_studies = list(df_screening.loc[df_screening.eligable == True,"pmc"])
result_table = pd.DataFrame(columns = ['pmc', 'ratio_mortality', 'lower_confidence_bound_mortality', 'upper_confidence_bound_mortality', 'p_value_mortality', 'ratio_value_recovery', 'lower_confidence_bound_recovery', 'upper_confidence_bound_recovery', 'p_value_recovery','sample_size_remdesivir', 'sample_size_control', 'placebo_controlled'])
result_table['pmc'] = np.array(eligable_studies)
result_table.to_csv("result_table.csv", index = False, sep = ";")
print("{} studies were found to be eligable".format(len(eligable_studies)))
for i in range(len(eligable_studies)):
    print("https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(eligable_studies[i]))

7 studies were found to be eligable
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7262788/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8757570/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8863204/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8279143/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7190303/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7442954/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8439621/


## Extract Relevant Data
Open the file result-table.csv. The separator is semicolon. Open the studies one-by-on by clicking on the corresponding url in the output of the cell above. Please follow the protocol below when you record your findings. If a meassure is missing, leave the cell blank.
### Ratio Mortality
Type the value of the hazard- or rate-ratio for mortality. You might have to look into an appendix mentioned in the paper. If mortality-ratios are reported for different time periods, choose the latest. If a study includes hospitalization or death don't include this study, because death itself is the outcome meassure.
### lower confidence bound mortality
The confidence-intervall is usually reported after the ratio. E.g. [95% CI 0·81–1·61] - the example is from an actual paper. You can see that the decimal point is missing. Please type the lower confidence bound (the left one) into the result_table. In the example it would be 0.81.
### upper confidence bound mortality
Equivalently type the upper confidence bound (the right one) into the table. In the example it would be 1.61.
### p-value mortality
Type the p-value into the result-table. It is usually documented right next to the confidence interval. If it isn't you have to look a bit in the results section. If there is only a confidence interval reported, leave the column p-value empty.
### Ratio Recovery
Different studies use different recovery-meassures. You should only report one. If a ratio is reported for time to recovery use that one, if not, look if there is time to improvement or aleviation of symptoms or an equivalent meassure.
Type the value of the hazard- or rate-ratio or odds-ratio. You might have to look into an appendix mentioned in the paper. If mortality-ratios are reported for different time periods, choose the latest.
### lower confidence bound Recovery
The confidence-intervall is usually reported after the ratio. E.g. [95% CI 0·81–1·61] - the example is from an actual paper. You can see that the decimal point is missing. Please type the lower confidence bound (the left one) into the result_table. In the example it would be 0.81.
### upper confidence bound Recovery
Equivalently type the upper confidence bound (the right one) into the table. In the example it would be 1.61.
### p-value Recovery
Type the p-value into the result-table. It is usually documented right next to the confidence interval. If it isn't you have to look a bit in the results section. If there is only a confidence interval reported, leave the column p-value empty.
### sample-sizes
Type the sample-sizes for the remdesivir group and the control-group seperately.
### placebo
check if the control group recevided a placebo or not. If the control-group received a placebo type 1 into the column and else type 0.
### save the file.
